In [1]:
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F  
import math
import os
import time
import inspect


In [16]:

class CasualSelfAttention(nn.Module):   #Relationships between tokens (via self-attention)

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # regularlization
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        # not really a bias, more of a mask, but following the OpenAi/HF naming
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size)).view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()  # batch, sequence length, embeding size
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # Every other student compares that question to their Key (what they know).
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # question a student is asking
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)  # Each student gives their Value (knowledge), weighted by how well their Key matched the Query.
        # attention
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))  #(B, n_head, T, head_dim) @ (B, n_head, head_dim, T) ==> (B, n_head, T, T)
        # Determines how relevant each other token is
        # q: (2, 4, 5, 64)
        # kᵀ: (2, 4, 64, 5)
        # att = q @ kᵀ: (2, 4, 5, 5)
        att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        # shape will become (1, 1, T, T)
        # [[[1, 0, 0, 0],
        #   [1, 1, 0, 0],
        #   [1, 1, 1, 0],
        #   [1, 1, 1, 1]]]
        att = F.softmax(att, dim=-1)
        # Ensures focus is on the most relevant tokens
        # softmax does not change the dimensionality; 
        # it just normalizes the values along the specified dimension (dim=-1).
        y = att @ v  # (B, n_head, T, head_dim):
        # Gathers contextualized information from those tokens
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        # (B, T, C)
        y = self.c_proj(y)
        return y
    
class MLP(nn.Module):  #A two-layer feedforward network : Learns token-level transformations

    def __init__(self, config):
        super().__init__()
        """
        Applies to each token independently
        Self-attention | Multiple tokens (sequence-wise) | Mixes information across positions
        MLP | One token (feature-wise) | Refines how info is represented per token
        Self-attention: “How does this token relate to other tokens in the sentence?”
        MLP: “How do I better express or transform the features of this one token?”
        """
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)  #Projects to higher dimension (richer representation)
        self.gelu = nn.GELU(approximate='tanh')    #Adds non-linearity (helps model complex patterns)
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)   #Projects back to original embedding size
    
    def forward(self, x):  
        x = self.c_fc()
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        """
        Layernorm: computes stats over C (embedding dim) for each token
        """
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CasualSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)
    
    def forward(self, x):  # residual connections - Helps training deeper models
        """
        Without residuals: every author rewrites the whole story from scratch.
        With residuals: each author edits or improves what the previous one wrote.
        Eases Gradient Flow:makes training deeper networks more stable and faster.
        Helps with Identity Mapping: If a layer isn’t useful, the model can easily learn to do nothing:  This prevents deeper layers from hurting performance and helps optimization.
        Encourages Incremental Learning: it can just refine or tweak it.
        """
        x = x + self.attn(self.ln_1(x))
        x = x + self.attn(self.ln_2(x))
        return x

@dataclass
class GPTConfig:
    """
    Head size = how deeply each attention head can understand the input.
    Depth (larger head size = deeper understanding per head)

    Number of heads = how broadly the model explores different relationships.
    Have enough heads to capture diverse relationships (semantic, positional, etc.)
    Breadth (more heads = more perspectives)    
    """
    block_size: int = 1024  # max sqeunce length (T)
    vocab_size: int = 50257  # 65 number of tokes: 50000 BPE merges + 256 bytes tokens + <|endoftext|>
    n_layer: int = 12  # number of transformer block or stacked layers
    n_head: int = 12  # number of attention heads
    n_embd: int = 768 # embedding dimension
    
class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config

        """ 
        input x -> (B, T)
        wte -> (B, T, C(n_embd)) (x)
        wpe -> (B, T, C(n_embd)) (position)
        """

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),  #word token embeddings
            wpe = nn.Embedding(config.block_size, config.n_embd),  # position embeddings
            # “The word ‘dog’ at position 1 is not the same as ‘dog’ at position 100.”
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),  # transformer layers
            ln_f = nn.LayerNorm(config.n_embd),  # final layer
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)  # output layer

    def forward(self, idx):
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of the length {T}, block size of {B}"
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device)  # shape (T)
        pos_emb = self.transformer.wpe(pos) # pos embeddings of shape (T, n_embd)
        tok_emb = self.transformer.wte(idx) # token embeddings (B, T, n_embd)
        x = tok_emb + pos_emb  # adding these
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)  #(B, T, vocab_size)
        return logits
    
    # load parameters
    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024),
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280),
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600),
        }[model_type]
        config_args['vocab_size'] = 50257  # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024  # always 1024 for GPT model checkpoints

        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()  #.state_dict() that returns a dictionary of all learnable parameters and buffers in the model.
        sd_keys = sd.keys()  # key is .weight / .bias etc..
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')]

        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]  #This removes any keys that end with .attn.masked_bias  These are buffer entries (not trainable parameters) often used for attention masks 
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')]
        transposed = ['attn.c_attn.weight','attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])
        return model

In [5]:
# -------------------
model = GPT.from_pretrained('gpt2')
print("didn't crash yay")

loading weights from pretrained gpt: gpt2


d:\develop\mit\env\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\konno\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, in

didn't crash yay


In [6]:
# Device detection
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'
print(f"using device: {device}")


using device: cpu


In [ ]:
num_return_sequences = 5
max_length = 10
model = GPT(GPTConfig())
model.eval()
model.to(device)

# prefix tokens
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to(device)
# print("length of tokens", len(tokens), tokens)
print("x", x.size())  #=>(5, 8)

# generate right now.  (B, T)  B = 5, T = 8
torch.manual_seed(66)
torch.cuda.manual_seed(66)
# while x.size(1) < max_length:
for i in range(1):
    with torch.no_grad():
        logits = model(x)  #[5, 8, 50257]  (B, T, vocab_size)
        print("logit shape", logits.shape)
        logits = logits[:, -1, :]  #[5, 50257]  (B, vocab_size)
        # only need the last token’s logits to predict the next word.
        # Get logits for last token to sample next
        print("after logit shape", logits.shape)
        probs = F.softmax(logits, dim=-1) #[5, 50257]  (B, vocab_size)
        print("probs", probs.shape)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)  #torch.Size([5, 50]) torch.Size([5, 50])
        # topk_probs[i] gives the top 50 probabilities for the i-th example in the batch.
        # topk_indices[i] gives the corresponding token indices (in your vocab of 50257 tokens).
        print("topk_probs", topk_probs.shape, topk_indices.shape)
        ix = torch.multinomial(topk_probs, 1)  #[5, 1]
        # Get the top-k most probable tokens.
        # Sample one from them randomly, but with preference given to higher-probability tokens.
        # This avoids always picking the most likely token (which is greedy).
        # Adds creativity and diversity to the generated output.
        print("ix", ix.shape)
        xcol = torch.gather(topk_indices, -1, ix)  #[5, 1]
        print("xcol", xcol.shape)
        x = torch.cat((x, xcol), dim=1)  #[5, 9]
        print("x==", x.shape)

x torch.Size([5, 8])
logit shape torch.Size([5, 8, 50257])
after logit shape torch.Size([5, 50257])
probs torch.Size([5, 50257])
topk_probs torch.Size([5, 50]) torch.Size([5, 50])
ix torch.Size([5, 1])
xcol torch.Size([5, 1])
x== torch.Size([5, 9])


In [ ]:
a = torch.tril(torch.ones(8, 8)).view(1, 1, 8, 8)
a

In [ ]:
# Config
batch_size = 2
seq_length = 4
vocab_size = 100
embedding_dim = 8
block_size = 16  # max sequence length

# Sample input
token_ids = torch.tensor([
    [5, 8, 2, 3],   # example 1
    [7, 6, 1, 0]    # example 2
])  # Shape: (2, 4)

# Position indices (0 to T-1)
position_ids = torch.arange(seq_length)  # Shape: (4,)

# Embedding layers
wte = nn.Embedding(vocab_size, embedding_dim)  # token embedding
wpe = nn.Embedding(block_size, embedding_dim)  # position embedding

# Apply embeddings
tok_emb = wte(token_ids)           # (2, 4, 8)
pos_emb = wpe(position_ids)        # (4, 8)
x = tok_emb + pos_emb              # (2, 4, 8) — broadcasted addition

In [ ]:
tok_emb[0]

In [ ]:
pos_emb[1]

In [ ]:
x[0]